Experiment with OpenCV for finding the pool table edges within an image

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import numpy.polynomial.polynomial as poly
import math
from Utils import *
import copy
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from ipycanvas import Canvas

In [ ]:
# Load the calibration and test image
dirCalImages = "AmcrestCamera/Checkerboards"

LoadCalibration(dirCalImages)
    
files, dir = FindImageFilesAndDir(subdir=dirCalImages)
filenameTest = files[0]

# Hack-in some ROI tweeking (xywh)
roiNew = (275, 976, 1461, 2280)
SetCalibrationROI(roiNew)

imageTest = cv.imread(filenameTest)
imageTestClipped, imageTestUnclipped = CalibrateImage(imageTest)

ImgShow([imageTestUnclipped, imageTestClipped], 100)

In [ ]:
# Load the test image
dirImages = "AmcrestCamera/Session5"
filename, _ = FindImageFileAndDir(dirImages, "table44.jpg")

print(f'Loading and calibrating file {filename}')
imageOriginal = cv.imread(filename)
imageOriginalClipped, imageOriginalUnclipped = CalibrateImage(imageOriginal)

TX_Load(dirImages)
            
ImgShow([imageOriginalUnclipped, imageOriginalClipped], 100)

imageOriginal = imageOriginalClipped

In [ ]:
# Load the saved transform
TX_Load(dirImages)

# Build a mask for the playable area
imageFelt = TX_MaskPlayableArea(imageOriginal)

ImgShow([imageOriginal, imageFelt], 100)

In [ ]:
# Find the balls, rough first pass

ballDiameter2D = int(2.25 * 100)
ballRadius2D = round(ballDiameter2D / 2)
print(f'ballRadius2D={ballRadius2D} ballDiameter2D={ballDiameter2D}')

def Distance(pts):
    dx = pts[1][0] - pts[0][0]
    dy = pts[1][1] - pts[0][1]
    sumOfSquares = dx*dx + dy*dy
    return math.sqrt(sumOfSquares)


imageFeltG = cv.cvtColor(imageFelt, cv.COLOR_BGR2GRAY)
imagePlaneGEq = cv.equalizeHist(imageFeltG)

# Hough Circles. 
imageInput = imagePlaneGEq
print("imageInput.shape = {0}".format(imageInput.shape))

def FindBalls(imageInput, bAltMode, blur, p1, p2, minDist, minRadius, maxRadius):    
    if bAltMode:
        mode = cv.HOUGH_GRADIENT_ALT
    else:
        mode = cv.HOUGH_GRADIENT

    if blur > 0:
        imageBlur = cv.GaussianBlur(imageInput, (blur,blur), cv.BORDER_DEFAULT)
    else:
        imageBlur = imageInput
        
    hcircles = cv.HoughCircles(imageBlur, mode, minDist=minDist, dp=1,
                               param1=p1, param2=p2,
                               minRadius=minRadius, maxRadius=maxRadius)

    # Create an array of points and radii
    balls2D = []
    ballsN = []
    imageBalls = imageOriginal.copy()
    if hcircles is not None:
        hcircles = np.uint16(np.around(hcircles))
        for hcirc in hcircles[0, :]:
            # We have the point in native space
            x, y, r = hcirc
            ptN = [x,y]
            
            # Check the transformed diameter to see if it is sized like a ball (left to right span)
            ballSidesN = [ [x-r, y], [x+r, y] ]
            ballSides2D = TX_ToFlatPoints(ballSidesN)
            dia2D = Distance(ballSides2D)
            diff = abs(ballDiameter2D - dia2D)
            #print(f'diff={diff}')
            #if diff > 35:
            #    continue    # Skip this ball, it is not the right size in 2D space
            
            cv.circle(imageBalls, ptN, r, Color(), 3)
            
            ballN = hcirc
            ballsN.append(ballN)
            
            pt2D = TX_ToFlatPoint(ptN)
            ball2D = [round(pt2D[0]), round(pt2D[1]), r]
            balls2D.append(ball2D)
            #print(f'  ball={ball}')
            
    return balls2D, ballsN, imageBalls


print(f'image.shape={imageInput.shape}')
balls2D = []
ballsN = []
def OnGui(bAltMode, blur, p1, p2, minRadius, maxRadius):
    # Blur size must be odd
    blur2 = blur
    if 0 == (blur2 % 2):
        blur2 -= 1
    minDist = minRadius * 2
    global balls2D
    global ballsN
    global imageBallsN
    balls2D, ballsN, imageBallsN = FindBalls(imageInput, bAltMode, blur2, p1, p2, minDist, minRadius, maxRadius)
    ImgShow([imageInput, imageBallsN],150)            
    print("balls count 2D = {0}".format(len(balls2D)))
    print("balls count N  = {0}".format(len(ballsN)))
    ImgShow([imageBallsN],350)            
    
w = interactive(OnGui,  bAltMode = widgets.Checkbox(value = False),
                        blur=widgets.IntSlider(min=0, max=71, value=4, step=1),
                        p1=widgets.IntSlider(min=1, max=500, value=35, step=1),
                        p2=widgets.IntSlider(min=1, max=500, value=25, step=1),
                        minRadius=widgets.IntSlider(min=1, max=100, value=18, step=1),
                        maxRadius=widgets.IntSlider(min=1, max=400, value=37, step=1),
                       )
display(w)

In [ ]:
# Color Space investigation for ball edge detection

imageInput = imageOriginal.copy()

def Merge(images):
    for idx, image in enumerate(images):
        if 0 == idx:
            imageMerged = image
        else:
            imageMerged = np.add(imageMerged, image)
            imageMerged = np.clip(imageMerged, 0, 255)
    return imageMerged


def MultiPlaneSobel(image, sobelD, sobelK):
    planes = image.shape[2]
    images = []
    for i in range(0, planes):
        imagePlane = image[:, :, i]
        imageSobel = cv.Sobel(imagePlane, ddepth=cv.CV_8UC1, dx=sobelD, dy=sobelD, ksize=sobelK)
        images.append(imageSobel)
    imageEdges = Merge(images)
    return imageEdges


def MultiPlaneLaplacian(image, ksize):
    planes = image.shape[2]
    images = []
    for i in range(0, planes):
        imagePlane = image[:, :, i]
        imageLaplace = cv.Laplacian(imagePlane, ddepth=cv.CV_8UC1, ksize=ksize)
        images.append(imageLaplace)
    imageEdges = Merge(images)
    return imageEdges



def ProcessColorImage(imageClr, sobelD, sobelK, laplacianKernel, bShowCircles, p1, p2, minRadius, maxRadius): 
    images = [imageClr]
    
    # Group by Sobel then Laplace
    imagesS = []
    imagesL = []
    for i in range(0, imageClr.shape[2]):
        # Get one plane
        imagePlane = imageClr[:, :, i]        
        
        # Do Sobel and Laplacian for it
        imageSobel = cv.Sobel(imagePlane, ddepth=cv.CV_8UC1, dx=sobelD, dy=sobelD, ksize=sobelK)
        imagesS.append(imageSobel)        
        imageLaplace = cv.Laplacian(imagePlane, ddepth=cv.CV_8UC1, ksize=laplacianKernel)
        imagesL.append(imageLaplace)     
        
    # Do the multi-plane for both Sobel and Laplacian
    imagesS.append(MultiPlaneSobel(imageClr, sobelD, sobelK))
    imagesL.append(MultiPlaneLaplacian(imageClr, ksize=laplacianKernel))
    
    images.extend(imagesS)
    images.extend(imagesL)
        
    if not bShowCircles:
        return images
    
    # Circles
    for idx, img in enumerate(images):
        if 3 == len(img.shape) and 1 != img.shape[2]:
            continue
        
        hcircles = cv.HoughCircles(img, cv.HOUGH_GRADIENT, minDist=maxRadius*2, dp=1,
                        param1=p1, param2=p2,
                        minRadius=minRadius, maxRadius=maxRadius)
        
        img = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
        
        # Draw the circles found
        clrCirc = (0,255,0)
        if hcircles is not None:
            hcircles = np.uint16(np.around(hcircles))
            hcircs = hcircles[0, :]
            for hcirc in hcircs:
                # We have the point in native space
                x, y, r = hcirc
                #print(f'x, y, r = {x},{y},{r}')
                ptN = [x,y]                    
                cv.circle(img, ptN, r, clrCirc, 1)
                
        images[idx] = img
    return images
    
    
images = [] # make a global
def OnGui(sobelDK, laplacianKernel, ballNum, bShowCircles, p1, p2, minRadius, maxRadius): 
    # Parse sobelDK
    tokens = sobelDK.split('/')
    sobelD = int(tokens[0])
    sobelK = int(tokens[1])
    
    if laplacianKernel % 2 == 0:
        laplacianKernel += 1
        
    ballIndex = ballNum - 1
           
    global images
    images = []
    for idx, ballN in enumerate(ballsN):    
        
        # Skip this ball?
        if ballIndex > -1 and idx != ballIndex:
            continue
        
        # extract just the circle/rect from the image, leave a padding border
        x,y,r = ballN
        padding = round(r / 2)
        r += padding
        imageBallOnly = imageInput[y-r-1:y+r+1, x-r-1:x+r+1, :]
        imageBGR = imageBallOnly.copy()    
        
        # We will edge-find in all these images, single-plane only
        images.extend(ProcessColorImage(imageBGR, sobelD, sobelK, laplacianKernel, bShowCircles, p1, p2, minRadius, maxRadius))
        images.extend(ProcessColorImage(cv.cvtColor(imageBGR, cv.COLOR_BGR2HSV), sobelD, sobelK, laplacianKernel, bShowCircles, p1, p2, minRadius, maxRadius))
        images.extend(ProcessColorImage(cv.cvtColor(imageBGR, cv.COLOR_BGR2HLS), sobelD, sobelK, laplacianKernel, bShowCircles, p1, p2, minRadius, maxRadius))
         
    
    ImgShow(images, 210, cols=9)
    # <end OnGui>

sobelDKOptions = [  "1/1", "1/3", "1/5", "1/7",
                    "2/1", "2/3", "2/5", "2/7",
                    "3/5", "3/7",
                    "4/5", "4/7",
                    "5/7",
                    "6/7"]
w = interactive(OnGui,
                sobelDK=widgets.Dropdown(options=sobelDKOptions, value="1/1"),
                laplacianKernel=widgets.IntSlider(min=1, max=31, value=2, step=1),
                ballNum=widgets.IntSlider(min=0, max=16, value=15, step=1),                                          
                bShowCircles = widgets.Checkbox(value = True),                
                p1=widgets.IntSlider(min=1, max=500, value=35, step=1),
                p2=widgets.IntSlider(min=1, max=500, value=25, step=1),
                minRadius=widgets.IntSlider(min=1, max=100, value=10, step=1),
                maxRadius=widgets.IntSlider(min=1, max=400, value=24, step=1)
                )
display(w)

In [ ]:
# Color Space investigation for ball edge detection

imageInput = imageOriginal.copy()

def Merge(images):
    for idx, image in enumerate(images):
        if 0 == idx:
            imageMerged = image
        else:
            imageMerged = np.add(imageMerged, image)
            imageMerged = np.clip(imageMerged, 0, 255)
    return imageMerged


def MultiPlaneSobel(image, sobelD, sobelK):
    planes = image.shape[2]
    images = []
    for i in range(0, planes):
        imagePlane = image[:, :, i]
        imageSobel = cv.Sobel(imagePlane, ddepth=cv.CV_8UC1, dx=sobelD, dy=sobelD, ksize=sobelK)
        images.append(imageSobel)
    imageEdges = Merge(images)
    return imageEdges


def MultiPlaneLaplacian(image, ksize):
    planes = image.shape[2]
    images = []
    for i in range(0, planes):
        imagePlane = image[:, :, i]
        imageLaplace = cv.Laplacian(imagePlane, ddepth=cv.CV_8UC1, ksize=ksize)
        images.append(imageLaplace)
    imageEdges = Merge(images)
    return imageEdges



def ProcessColorImage(imageClr, sobelD, sobelK, laplacianKernel, bShowCircles, p1, p2, minRadius, maxRadius): 
    images = [imageClr]
    
    # Group by Sobel then Laplace
    imagesS = []
    imagesL = []
    for i in range(0, imageClr.shape[2]):
        # Get one plane
        imagePlane = imageClr[:, :, i]        
        
        # Do Sobel and Laplacian for it
        imageSobel = cv.Sobel(imagePlane, ddepth=cv.CV_8UC1, dx=sobelD, dy=sobelD, ksize=sobelK)
        imagesS.append(imageSobel)        
        imageLaplace = cv.Laplacian(imagePlane, ddepth=cv.CV_8UC1, ksize=laplacianKernel)
        imagesL.append(imageLaplace)     
        
    # Do the multi-plane for both Sobel and Laplacian
    imagesS.append(MultiPlaneSobel(imageClr, sobelD, sobelK))
    imagesL.append(MultiPlaneLaplacian(imageClr, ksize=laplacianKernel))
    
    images.extend(imagesS)
    images.extend(imagesL)
        
    if not bShowCircles:
        return images
    
    # Circles
    for idx, img in enumerate(images):
        if 3 == len(img.shape) and 1 != img.shape[2]:
            continue
        
        hcircles = cv.HoughCircles(img, cv.HOUGH_GRADIENT, minDist=maxRadius*2, dp=1,
                        param1=p1, param2=p2,
                        minRadius=minRadius, maxRadius=maxRadius)
        
        img = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
        
        # Draw the circles found
        clrCirc = (0,255,0)
        if hcircles is not None:
            hcircles = np.uint16(np.around(hcircles))
            hcircs = hcircles[0, :]
            for hcirc in hcircs:
                # We have the point in native space
                x, y, r = hcirc
                #print(f'x, y, r = {x},{y},{r}')
                ptN = [x,y]                    
                cv.circle(img, ptN, r, clrCirc, 1)
                
        images[idx] = img
    return images
    
    
images = [] # make a global
def OnGui(sobelDK, laplacianKernel, ballNum, bShowCircles, p1, p2, minRadius, maxRadius): 
    # Parse sobelDK
    tokens = sobelDK.split('/')
    sobelD = int(tokens[0])
    sobelK = int(tokens[1])
    
    if laplacianKernel % 2 == 0:
        laplacianKernel += 1
        
    ballIndex = ballNum - 1
           
    global images
    images = []
    for idx, ballN in enumerate(ballsN):    
        
        # Skip this ball?
        if ballIndex > -1 and idx != ballIndex:
            continue
        
        # extract just the circle/rect from the image, leave a padding border
        x,y,r = ballN
        padding = round(r / 2)
        r += padding
        imageBallOnly = imageInput[y-r-1:y+r+1, x-r-1:x+r+1, :]
        imageBGR = imageBallOnly.copy()    
        
        # We will edge-find in all these images, single-plane only
        images.extend(ProcessColorImage(imageBGR, sobelD, sobelK, laplacianKernel, bShowCircles, p1, p2, minRadius, maxRadius))
        images.extend(ProcessColorImage(cv.cvtColor(imageBGR, cv.COLOR_BGR2HSV), sobelD, sobelK, laplacianKernel, bShowCircles, p1, p2, minRadius, maxRadius))
        images.extend(ProcessColorImage(cv.cvtColor(imageBGR, cv.COLOR_BGR2HLS), sobelD, sobelK, laplacianKernel, bShowCircles, p1, p2, minRadius, maxRadius))
         
    
    ImgShow(images, 210, cols=9)
    # <end OnGui>

sobelDKOptions = [  "1/1", "1/3", "1/5", "1/7",
                    "2/1", "2/3", "2/5", "2/7",
                    "3/5", "3/7",
                    "4/5", "4/7",
                    "5/7",
                    "6/7"]
w = interactive(OnGui,
                sobelDK=widgets.Dropdown(options=sobelDKOptions, value="1/1"),
                laplacianKernel=widgets.IntSlider(min=1, max=31, value=2, step=1),
                ballNum=widgets.IntSlider(min=0, max=16, value=15, step=1),                                          
                bShowCircles = widgets.Checkbox(value = True),                
                p1=widgets.IntSlider(min=1, max=500, value=35, step=1),
                p2=widgets.IntSlider(min=1, max=500, value=25, step=1),
                minRadius=widgets.IntSlider(min=1, max=100, value=10, step=1),
                maxRadius=widgets.IntSlider(min=1, max=400, value=24, step=1)
                )
display(w)

In [ ]:
# Clever Ball Finding
#  We used the previous HoughCircles operation as the 'gross' find. Now we 
#  try to narrow it down separately for each ball. We will more closely limit
#  the allowable radii.

imageInput = imageOriginal.copy()   

def CalcBallRadiusAtPoint(ptN):
    # Transform point to 2D space
    x, y = TX_ToFlatPoint(ptN)
    
    # Build a cross centered on the ball in 2D space
    #  l,r,t,b
    r2D = int(round(2.25 * 100 / 2))
    ptsBox2D = [(x-r2D,y), (x+r2D,y), (x,y-r2D), (x,y+r2D)]
    
    # Transform the box back to N space
    l,r,t,b = TX_FromFlatPoints(ptsBox2D)
    
    # What is the radius? (width/2, height/2)
    rx = (Distance([l,r])) / 2
    ry = (Distance([t,b])) / 2
    
    # Average of the two
    return (rx + ry) / 2

# x,y,r for each ball found in image N space
ballImages = []
ballPositionsL = []   # Local to ballImages
ballPositionsN = []   # Native coordinates

def OnGui(sobelDK, blur, p1, p2, radFuzz):   
    # Parse sobelDK
    tokens = sobelDK.split('/')
    sobelD = int(tokens[0])
    sobelK = int(tokens[1])
    
    # Blur size must be odd
    blur2 = blur
    if 0 == (blur2 % 2):
        blur2 -= 1     
        
    images = []
    circCounts = []
    for ballN in ballsN:    
        # extract just the circle/rect from the image, leave a padding border
        x,y,r = ballN
        padding = round(r / 2)
        r += padding
        top = y-r-1
        left= x-r-1
        imageBallOnly = imageInput[top:y+r+1, left:x+r+1, :]
        imageBallOnly = imageBallOnly.copy()
        ballImages.append(imageBallOnly)
        
        if blur2 > 0:
            imageBlur = cv.GaussianBlur(imageBallOnly, (blur2,blur2), cv.BORDER_DEFAULT)
        else:
            imageBlur = imageBallOnly
            
        # Edge detection
        imageEdgesClr = cv.Sobel(imageBlur, ddepth=cv.CV_8UC1, dx=sobelD, dy=sobelD, ksize=sobelK)
        imageEdgesGray = cv.cvtColor(imageEdgesClr, cv.COLOR_BGR2GRAY)       
        
        radiusExpected = CalcBallRadiusAtPoint((x,y))
        #print(f'pt={x},{y} radiusExpected={radiusExpected}')
        minRadius=int(radiusExpected-radFuzz)
        maxRadius=int(radiusExpected+radFuzz)
        hcircles = cv.HoughCircles(imageEdgesGray, cv.HOUGH_GRADIENT, minDist=2*radiusExpected, dp=1,
                                param1=p1, param2=p2,
                                minRadius=minRadius, maxRadius=maxRadius)
        
        # Draw the circles found
        clrCirc = (0,255,0)
        circCount = 0
        imageBallWithCirc = imageBallOnly.copy()
        if hcircles is not None:
            hcircles = np.uint16(np.around(hcircles))
            hcircs = hcircles[0, :]
            for hcirc in hcircs:
                # We have the point in native space
                x, y, r = hcirc
                ptN = [x,y]
                
                cv.circle(imageBallWithCirc, ptN, r, clrCirc, 1)
                #print(f'pt={x},{y} r={r}')
                circCount += 1
                
                ballPositionsL.append( (x, y, r) )
                ballPositionsN.append( (left+x, top+y, r) )
        
        circCounts.append(circCount)
        
        images.append(imageBallWithCirc)
        images.append(imageEdgesGray)
            
        # <end for each ball>
            
    print(f'circCounts={circCounts}')
    
    ImgShow(images, 150, cols=8)

sobelDKOptions = [  "1/1", "1/3", "1/5", "1/7",
                    "2/1", "2/3", "2/5", "2/7",
                    "3/5", "3/7",
                    "4/5", "4/7",
                    "5/7",
                    "6/7"]

w = interactive(OnGui,  sobelDK=widgets.Dropdown(options=sobelDKOptions, value="1/1"),
                        blur=widgets.IntSlider(min=0, max=71, value=2, step=1),
                        p1=widgets.IntSlider(min=1, max=500, value=35, step=1),
                        p2=widgets.IntSlider(min=1, max=500, value=10, step=1),
                        radFuzz=widgets.IntSlider(min=1, max=15, value=3, step=1)
                       )
display(w)


In [ ]:
# numpy
a = np.array([0,1,2,3,4,5,6,7,8,9,8,7,6,5,4,3,2,1,0])
d = np.diff(a)
print(a)
print(d)

b = np.array([0,0,1,2,1,0,0,1,1,1,0,1,2,3,2,0])
filter = np.array([1,1,1])
c = np.convolve(b, filter)
print("b", b)
print("c", c)
c = c[1:len(c)-1]
print("c", c)

def WindowAverage(arr, width):    
    filter = np.ones(width, dtype=np.int32)
    c = np.convolve(arr, filter)
    c = c[1:len(c)-1]
    return c

print("c", WindowAverage(b, 3))


m = np.amax(a)
print(f'm={m}')


In [ ]:

# Identify ball colors

def BGR2HtmlColor(bgr):
    b,g,r = bgr
    b = '{:02X}'.format(b)
    g = '{:02X}'.format(g)
    r = '{:02X}'.format(r)
    return "#" + r + g + b
    
clr = (255,22, 0)
print(BGR2HtmlColor(clr))
clr = (255,0, 254)
print(BGR2HtmlColor(clr))
    
class BallDef:
    def __init__(self, num, bgrDisplay, bgrMatch):
        self.num = num
        self.bSolid = num < 9
        self.bgrDisplay = bgrDisplay
        self.bgrMatch = bgrMatch
        
    def ColorDistance(self, clr):
        sumOfSquares = 0
        for i in range(0,3):
            diff = clr[i] - self.bgrMatch[i]
            sumOfSquares += diff * diff
        return math.sqrt(sumOfSquares)
    
    def Desc(self):
        typ = "solid" if self.bSolid else "stripe"
        desc = str(self.num) + " " + typ
        return desc
    
    def DisplayColorHTML(self):
        return BGR2HtmlColor(self.bgrDisplay)
    
    def DisplayColorBGR(self):
        return self.bgrDisplay
    
    def IsSolid(self):
        return self.bSolid
    
    def Num(self):
        return self.num

clrYel = (0,255,255)
clrRed = (0,0,255)
clrGrn = (70,130,23)
clrBlue= (255,0,0)
clrPurple= (95,45,95)
clrOrange= (11,107,255)
clrMaroon= (30,34,105)
clrBlk = (10,10,10)
clrWhite=(220, 220, 220)

ballDefs = []
ballDefs.append(BallDef(0, clrWhite, (74, 142, 181)))

ballDefs.append(BallDef(1, clrYel,      (255, 42, 212)))
ballDefs.append(BallDef(2, clrBlue,     (53, 103, 108)))
ballDefs.append(BallDef(3, clrRed,      (29, 41, 29)))
ballDefs.append(BallDef(4, clrPurple,   (29, 37, 29)))
ballDefs.append(BallDef(5, clrOrange,   (142, 51, 129)))
ballDefs.append(BallDef(6, clrGrn, (45, 38, 69)))
ballDefs.append(BallDef(7, clrMaroon, (70, 101, 82)))
ballDefs.append(BallDef(8, clrBlk, (42, 64, 57)))

ballDefs.append(BallDef(9, clrYel, (118, 43, 145)))
ballDefs.append(BallDef(10, clrBlue, (20, 45, 50)))
ballDefs.append(BallDef(11, clrRed, (23, 39, 73)))
ballDefs.append(BallDef(12, clrPurple, (62, 46, 62)))
ballDefs.append(BallDef(13, clrOrange, (149, 102, 155)))
ballDefs.append(BallDef(14, clrGrn, (22, 40, 54)))
ballDefs.append(BallDef(15, clrMaroon,     (31, 34, 54)))

class Ball:
    def __init__(self, pt, ballDef):
        self.pt = pt
        self.ballDef = ballDef
        print(f'New Ball #{ballDef.Desc()} at {pt}')
        
    def Pos(self):
        return self.pt
    
    def GetBallDef(self):
        return self.ballDef

# Scan all ball def's for the ball that is closest in color
def FindBallDef(clr):
    minDistIdx = -1
    minDistVal = 10E10
    for idx, ballDef in enumerate(ballDefs):
        dist = ballDef.ColorDistance(clr)
        if dist < minDistVal:
            minDistVal = dist
            minDistIdx = idx
            
    return ballDefs[minDistIdx]
        
            
    
def WindowAverage(arr, width):    
    filter = np.ones(width)
    arr = np.ravel(arr)
    c = np.convolve(arr, filter)
    c = c[1:len(c)-1]
    return c
    
images = []
peaks = []
clrCirc = (255)
balls = []
for i, (imageBall, posL, posN) in enumerate(zip(ballImages, ballPositionsL, ballPositionsN)):
    if -1 == i:
        continue
    x,y,r = posL
    image = imageBall.copy()
    
    # Build a mask 
    imageMask = np.zeros(image.shape[:2], np.uint8)
    cv.circle(imageMask, [x,y], r, clrCirc, -1)    
    
    images.append(imageBall)    
    images.append(imageMask)
    
    peaksRGB = []
    for iClr in range(0,3):
        imgPlane = imageBall[:,:,iClr]
        hist = cv.calcHist([imgPlane],[0],imageMask,[256],[0,255])
        maxHist = int(np.amax(hist))
        maxHist = 255 if maxHist > 255 else maxHist
        #print(f'{i} maxHist={maxHist}')
        peaksRGB.append(maxHist)
                
        #colorName = ('b','g','r')   
        #if i == 9:      
            #plt.plot(hist, color=colorName[iClr])   
            #print(f'========= {i} {iClr} ==========')
            #print(f'hist.shape={hist.shape}')
            #print(hist)
    #print(f'{i} peaksRGB= ({peaksRGB[0]}, {peaksRGB[1]}, {peaksRGB[2]})')     
    peaks.append(peaksRGB)
    ballDef = FindBallDef(peaksRGB)
    
    x,y = TX_ToFlatPoint((posN[0], posN[1]))
    balls.append(Ball((x,y), ballDef))
    
plt.rcParams['figure.dpi'] = 20
plt.show()

ImgShow(images, 200, cols=10)


In [ ]:
# Draw the table
from math import pi
from ipycanvas import Canvas, Path2D

class Table:
    def __init__(self, widthInches, pixelWidth, canvas, xoffset=0, yoffset=0):
        self.canvas = canvas
        self.widthInches = widthInches
        self.clrWhite="#DCDCDC"
        self.clrFelt = "#0D4B5D"
        self.clrRail = "#1D5B6D"
        self.clrFeltShadow = "#0A3746"
        self.clrWood = "#5D420D"
        self.clrHole = "#000000"
        self.clrPocket = "#606060"
        self.xOffset = xoffset
        self.yOffset = yoffset
        
        # Dimensions
        self.width = pixelWidth
        self.woodBorder = pixelWidth // 15
        self.railBorder = pixelWidth // 15 // 2
        self.railBorderShadow = pixelWidth // 15 // 4 // 1        
        
        # The actual playing area is called 'the felt'. A ball can be located off the felt
        # if it is sitting just inside a pocket
        self.feltOffset = self.woodBorder + self.railBorder
        self.feltWidth = self.width - (2 * self.feltOffset)
        self.feltHeight = self.feltWidth * 2
        
        self.height = self.feltHeight + 2 * self.feltOffset
        
        dx2d, _ = TX_GetTargetDims()
        self.toTableFactor = self.feltWidth / dx2d
        self.ballRadius = 2.25 / 2.0 * 100 * self.toTableFactor
        self.pocketSizeCorner = 4.5 * 100 * self.toTableFactor
        self.pocketSizeSide = 5.0 * 100 * self.toTableFactor
        self.DrawTable()
        
    def _TableClip(self): 
        # Setup clipping path to give the table some pretty corners
        crnr = int(self.woodBorder / 1.75)
        self.canvas.begin_path()
        
        # Top-left
        self.canvas.move_to(0, crnr)
        self.canvas.line_to(crnr, 0)
        
        # Top-Right
        self.canvas.line_to(self.width-crnr, 0)
        self.canvas.line_to(self.width, crnr)
        
        # Bottom-Right
        self.canvas.line_to(self.width, self.height-crnr)
        self.canvas.line_to(self.width-crnr, self.height)
        
        # Bottom-Left
        self.canvas.line_to(crnr, self.height)
        self.canvas.line_to(0, self.height-crnr)
        
        # Close it
        self.canvas.line_to(0, crnr)        
        self.canvas.close_path()
        self.canvas.clip()
        
    
    def DrawTable(self):        
        self.canvas.save()
        self.canvas.translate(self.xOffset, self.yOffset)
        
        self._TableClip()
       
        # Wood
        self.canvas.fill_style = self.clrWood
        self.canvas.fill_rect(0, 0, width = self.width, height=self.height)
        
        # Draw felt color for both rails and felt area (we will add shaded rails after)
        self.canvas.fill_style = self.clrFelt
        self.canvas.fill_rect(self.woodBorder, self.woodBorder, width = self.feltWidth + 2*self.railBorder, height=self.feltHeight+ 2*self.railBorder)

        # Just for debug
        #self.canvas.fill_style = "#FF0000"
        #self.canvas.fill_rect(self.feltOffset,self.feltOffset, self.feltWidth, self.feltHeight)
        
        self._DrawRails()
        self._DrawPockets()
        self._DrawMarkers() 
        self.canvas.restore()
        return
    
        # Rail Shadows using gradients
        left = offset
        right = left + self.railBorderShadow
        gradL = self.canvas.create_linear_gradient(left,0, right,0, [(0, self.clrFeltShadow), (1, self.clrFelt)])    
        self.canvas.fill_style = gradL
        self.canvas.fill_rect(left, offset, width = self.railBorderShadow, height=self.height - 2 * offset)   
        
        right = self.width - offset
        left = right - self.railBorderShadow
        gradR = self.canvas.create_linear_gradient(left,0, right,0, [(0, self.clrFelt), (1, self.clrFeltShadow)])
        self.canvas.fill_style = gradR
        self.canvas.fill_rect(left, offset, width = self.railBorderShadow, height=self.height - 2 * offset)     
        
        top = offset
        bot = top + self.railBorderShadow 
        gradT = self.canvas.create_linear_gradient(0,top, 0,bot, [(0, self.clrFeltShadow), (1, self.clrFelt)])    
        self.canvas.fill_style = gradT
        self.canvas.fill_rect(offset, top, width = self.feltWidth, height=self.railBorderShadow)   
        
        bot = self.height - offset
        top = bot - self.railBorderShadow
        gradB = self.canvas.create_linear_gradient(0,top, 0,bot, [(0, self.clrFelt), (1, self.clrFeltShadow)])    
        self.canvas.fill_style = gradB
        self.canvas.fill_rect(offset, top, width = self.feltWidth, height=self.railBorderShadow)
                 
        self._DrawMarkers() 
        self.canvas.restore()
        
    def _DrawRails(self):
        cornerOffset = math.sqrt(self.pocketSizeCorner*self.pocketSizeCorner/2.0)
        sideOffset = math.sqrt(self.pocketSizeSide*self.pocketSizeSide/2.0)
        railLengthH = self.width - 2 * (self.woodBorder + cornerOffset)
        railLengthV = railLengthH * 0.975
        
        angleCornerDeg = 142
        angleSideDeg = 104
        
        # Top rail
        self.canvas.save()
        self.canvas.translate(self.woodBorder + cornerOffset, self.woodBorder)
        self._DrawRail(railLengthH, angleCornerDeg, angleCornerDeg)
        self.canvas.restore()
        
        # Right top
        self.canvas.save()
        self.canvas.translate(self.width - self.woodBorder, self.woodBorder + cornerOffset)
        self.canvas.rotate(pi/2.0)
        self._DrawRail(railLengthV, angleSideDeg, angleCornerDeg)
        self.canvas.restore()
        
        # Right bottom
        self.canvas.save()
        self.canvas.translate(self.width - self.woodBorder, self.height - self.woodBorder - railLengthV - cornerOffset)
        self.canvas.rotate(pi/2.0)
        self._DrawRail(railLengthV, angleCornerDeg, angleSideDeg)
        self.canvas.restore()
        
        # Left bottom
        self.canvas.save()
        self.canvas.translate(self.woodBorder, self.height - self.woodBorder - cornerOffset)
        self.canvas.rotate(pi*3.0/2.0)
        self._DrawRail(railLengthV, angleSideDeg, angleCornerDeg)
        self.canvas.restore()
        
        # Left top
        self.canvas.save()
        self.canvas.translate(self.woodBorder, self.woodBorder + railLengthV + cornerOffset)
        self.canvas.rotate(pi*3.0/2.0)
        self._DrawRail(railLengthV, angleCornerDeg, angleSideDeg)
        self.canvas.restore()
        
        # Bottom rail        
        self.canvas.save()
        self.canvas.translate(self.woodBorder + cornerOffset + railLengthH, self.height - self.woodBorder)
        self.canvas.rotate(pi)
        self._DrawRail(railLengthH, angleCornerDeg, angleCornerDeg)
        self.canvas.restore()
    
    def _Deg2Rad(self, deg):
        return deg / (360.0) * 2.0 * pi
    
    def _DrawRail(self, length, cutAngleDegA, cutAngleDegB):
        self.canvas.fill_style = self.clrRail
        h = self.railBorder
        acuteRadA = self._Deg2Rad(180 - cutAngleDegA)
        acuteRadB = self._Deg2Rad(180 - cutAngleDegB)
        dA = h / math.tan(acuteRadA)
        dB = h / math.tan(acuteRadB)
        #d = 0
        self.canvas.fill_polygon(
            [(0, 0),
             (length, 0),
             (length-dA, h),
             (dB, h),
             (0, 0)]
        )
        return
        
        
    def _DrawCornerPocket(self):
        # top-left corner pocket only, caller translates first
        
        # Define sizes (relative)     
        base =  int(self.pocketSizeCorner * 1)
        w = int(self.woodBorder / 1.75)
        
        # Black Hole
        self.canvas.fill_style = self.clrHole
        self.canvas.fill_arc(w, w, self.pocketSizeCorner, 0.0, pi/2.0)
        self.canvas.fill_arc(w//2, w//2, self.pocketSizeCorner, 0.0, pi/2.0)
        
        # Gray hole holder
        self.canvas.fill_style = self.clrPocket
        self.canvas.fill_polygon(
            [(0,0),
             (base,0),
             (base+self.woodBorder, self.woodBorder),
             (base+self.woodBorder-w, self.woodBorder),
             (base-w//3, w-w//3),
             (w, w), # center
             (w-w//3, base-w//3),
             (self.woodBorder, base+self.woodBorder-w),
             (self.woodBorder, base+self.woodBorder),
             (0, base),
             (0,0)]
        )
        
    def _DrawSidePocket(self):
        # Draw the left-side pocket, centered vertically aligned on left
        self.canvas.save()
        
        h =  self.pocketSizeSide * 1.0
        centerY = 0         
        base =  int(self.pocketSizeCorner * 1)
        d = int(self.woodBorder / 1.75)
        
        # Setup some clipping to make this easier
        self.canvas.begin_path()
        self.canvas.move_to(0, centerY-h)
        self.canvas.line_to(self.woodBorder * 1.2, centerY-h)
        self.canvas.line_to(self.woodBorder * 1.2, centerY+h)
        self.canvas.line_to(0, centerY+h)
        self.canvas.move_to(0, centerY-h)     
        self.canvas.close_path()
        self.canvas.clip()
        
        # Black Hole
        self.canvas.fill_style = self.clrHole
        self.canvas.fill_arc(-self.woodBorder/15, 0, self.pocketSizeCorner, -pi/2.0, pi/2.0)
        
        # Gray hole holder
        self.canvas.fill_style = self.clrPocket
        #self.canvas.fill_rect(offset, top, width = self.feltWidth, height=self.railBorderShadow)
        top = centerY-h
        bot = centerY+h
        self.canvas.fill_polygon(
            [(0, top),
             (self.woodBorder, top),
             (self.woodBorder, top+d),
             (d/2,top+d),
             (d/2,bot-d),
             (self.woodBorder, bot-d),
             (self.woodBorder, bot),
             (0, bot),
             (0, top)]
        )
        
        self.canvas.restore()
        
    def _DrawPockets(self):
        # Top Left
        self._DrawCornerPocket()        
        
        # Top Right
        self.canvas.save()
        self.canvas.translate(self.feltWidth + self.feltOffset*2, 0)
        self.canvas.rotate(pi/2.0)
        self._DrawCornerPocket()
        
        # Bottom Right
        self.canvas.translate(self.feltHeight + self.feltOffset*2, 0)   
        self.canvas.rotate(pi/2.0)
        self._DrawCornerPocket()     
        
        # Bottom Left
        self.canvas.translate(+self.feltWidth + self.feltOffset*2, 0)   
        self.canvas.rotate(pi/2.0)
        self._DrawCornerPocket()     
        
        self.canvas.restore()
        
        # Left Side
        self.canvas.save()
        self.canvas.translate(0, self.height/2)  
        self._DrawSidePocket()
        
        # Right side
        self.canvas.translate(self.feltWidth + self.feltOffset*2, 0)  
        self.canvas.rotate(pi)
        self._DrawSidePocket()
        self.canvas.restore()
        
    def _DrawMarkers(self):
        markerWidth = self.feltWidth / 4
        self.canvas.fill_style = '#DDDDDD'
        markerRadius = int(self.width / 175)
        
        # Horizontal markers
        offset = (self.woodBorder / 2)
        for i in range(1, 4):
            self.canvas.fill_circle(self.feltOffset + markerWidth * i, offset, markerRadius)
            self.canvas.fill_circle(self.feltOffset + markerWidth * i, self.height - offset, markerRadius)
        
        # Vertical markers
        for i in range(1, 8):
            if i == 4:
                continue
            self.canvas.fill_circle(offset, self.feltOffset + markerWidth * i, markerRadius)
            self.canvas.fill_circle(self.width - offset, self.feltOffset + markerWidth * i, markerRadius)
        
        
    # Convert a single value to table space from 2D space
    def TxVal(self, v):
        return v * self.toTableFactor
        
    def TxPt(self, pt):
        return (int(self.TxVal(pt[0])), int(self.TxVal(pt[1])))    
    
    def _DrawStripeBall(self, x,y, clrWeb, num):
        # Draw entire ball as color
        self._DrawSolidBall(x, y, clrWeb, num)
        
        # draw two sides in white (not the stripe)
        self.canvas.fill_style = self.clrWhite
        
        # Calculate the arc points from a zero reference point 
        th = pi / 5
        dx = self.ballRadius * math.cos(th)
        dy = self.ballRadius * math.sin(th)
        
        # Build the paths for the two sides
        s1 = "M {xstart} {ystart} A {rad},{rad} {rot} 0 1 {xend},{yend}".format(xstart=x-dx, ystart=y-dy, rad=self.ballRadius, rot=0, xend=x+dx, yend=y-dy)
        s2 = "M {xstart} {ystart} A {rad},{rad} {rot} 0 0 {xend},{yend}".format(xstart=x-dx, ystart=y+dy, rad=self.ballRadius, rot=0, xend=x+dx, yend=y+dy)
        self.canvas.fill(Path2D(s1))
        self.canvas.fill(Path2D(s2))            
        
    def _DrawSolidBall(self, x,y, clrWeb, num):
        self.canvas.fill_style = clrWeb
        self.canvas.fill_circle(x, y, self.ballRadius)
        
        # Draw the number in a white circle
        if 0 == num:
            return # Skip the cue ball
        self.canvas.fill_style = self.clrWhite
        self.canvas.fill_circle(x, y, self.ballRadius/2.5)
        self.canvas.fill_style = "#202020"
        px = int(self.ballRadius * 0.65)
        self.canvas.font = '{px}px sans-serif'.format(px=px)
        self.canvas.text_align = 'center'
        self.canvas.text_baseline = 'middle'
        self.canvas.fill_text(str(num), x, int(y + self.ballRadius/30))        
    
    def _DrawBallSelection(self, x,y):
        self.canvas.stroke_style = '#804040'
        self.canvas.stroke_circle(x, y, self.ballRadius)
        
    def DrawBall(self, ball, selected=False):
        # Adjust for table borders
        self.canvas.save()
        self.canvas.translate(self.xOffset + self.feltOffset, self.yOffset + self.feltOffset)
        
        pt = ball.Pos()
        x,y = self.TxPt(pt)
        
        clrWeb = ball.GetBallDef().DisplayColorHTML()
        num = ball.GetBallDef().Num()
        if ball.GetBallDef().IsSolid():
            self._DrawSolidBall(x,y, clrWeb, num)
        else:
            self._DrawStripeBall(x,y, clrWeb, num)
            
        if selected:
            self._DrawBallSelection(x,y)
            
        self.canvas.restore()


dx = 500
dy = int(dx * 2.05)
canvas = Canvas(width=dx+60, height=dy+60)

table = Table(44, dx, canvas, xoffset=10, yoffset=20)
    
for ball in balls:
    table.DrawBall(ball)
    
#b = Ball((0,0), ballDefs[10])
#table.DrawBall(b, selected=True)
#b = Ball((640,140), ballDefs[11])
#table.DrawBall(b, selected=True)

canvas

pockets

https://www.designerbilliards.co.uk/news/news/American-Pool-vs-English-Pool

Corner Pockets:
    Straight cut between 4.5 and 4.625 inches
    142 Degree cut in rails

Center Pockets: 
    Between 5 and 5.125 inches
    104 degree cut in rails

In [ ]:
# junk added on laptop to test git